In [ ]:
# Audio Summary Generation using watsonX Part - 2
## 1. Importing Libraries
import os
from dotenv import load_dotenv
import os

from genai.extensions.langchain import LangChainInterface
from genai.schemas import GenerateParams
from genai.credentials import Credentials

from langchain.document_loaders import PyPDFLoader
from langchain.indexes import VectorstoreIndexCreator
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter

from langchain import PromptTemplate
from langchain.document_loaders import TextLoader

In [ ]:
## 2. Setting up the API Key and Endpoint
load_dotenv()
api_key = os.getenv("GENAI_KEY", None)
api_url = os.getenv("GENAI_API", None)
creds = Credentials(api_key, api_endpoint=api_url)

In [ ]:
## 3. Setting up the model name and its parameters
model_id = "meta-llama/llama-2-70b-chat"
params = GenerateParams(
            decoding_method="greedy",
            max_new_tokens=100,
            min_new_tokens=10,
            temperature=0.7
        )

In [ ]:
llm=LangChainInterface(model=model_id, params=params, credentials=creds)

In [ ]:
## 4. Function to store open the txt file, convert it in to embeddings and store it in vector Database
def load_txt(): 
    pdf_name = 'transcriptios.txt'
    loaders = [TextLoader(pdf_name)]

    index = VectorstoreIndexCreator(
        embedding = HuggingFaceEmbeddings(model_name='all-MiniLM-L12-v2'), 
        text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
    ).from_loaders(loaders)

    return index

In [ ]:
## 5. Create the object of the function and call it
index = load_txt()

In [ ]:
## 6. Deffining the PROMPT TEMPLATE
prompt_template = """You are a furniture persnalization assistant. Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer. If you not find any information about the questions answer generally by your own knowledge.

{context}

Question: {question}
Answer:"""
PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

In [ ]:
## 7. Defing the chain with parameters
chain_type_kwargs = {"prompt": PROMPT}

qa = RetrievalQA.from_chain_type(llm, chain_type="stuff", retriever=index.vectorstore.as_retriever(), chain_type_kwargs=chain_type_kwargs)


In [ ]:
## 8. Test time
query = "Do you prefer second-hand or new furniture??"

print(qa.run(query))